# Quarterly Sales Tax Revenue Analysis

### [Data Source](https://tax.utah.gov/econstats/sales)

### Q2: April, May, June

In [1]:
import pandas as pd
import os
import numpy as np
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

In [2]:
# show all columns
pd.options.display.max_columns = None

In [3]:
# Removes misc characters from string
def replace(string):
    string = string.replace("&", "")
    string = string.replace("-", " ")
    string = string.replace("(", "")
    string = string.replace(")", "")
    string = string.replace("/", " ")
    string = string.replace(",", "")
    string = string.replace(".", "")
    string = string.replace(" ", "_")
    return string


### Load Q2 2020 sales tax revenue data

In [4]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2020-02-quarterly-sales.xlsx'
q2_2020 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
q2_2020.drop(q2_2020.tail(2).index,inplace=True)

# forward fill values from merged cells
q2_2020['County'] = q2_2020['County'].fillna(method='ffill')
q2_2020['Location Code'] = q2_2020['Location Code'].fillna(method='ffill')
q2_2020['City'] = q2_2020['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
q2_2020.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
q2_2020['Rounded Up'] = q2_2020['Rounded Up'].map(crosswalk)

q2_2020.head()


,County,Location Code,City,Economic Sector (NAICS Code),2020Q2,Rounded Up
0,Beaver County,1002.0,Beaver City,"AGRICULTURE, FORESTRY, FISHING & HUNTING (11)",400000.0,True
1,Beaver County,1002.0,Beaver City,UTILITIES (22),600000.0,True
2,Beaver County,1002.0,Beaver City,CONSTRUCTION (23),29553.0,False
3,Beaver County,1002.0,Beaver City,MANUFACTURING (31-33),1308568.0,False
4,Beaver County,1002.0,Beaver City,WHOLESALE TRADE-DURABLE GOODS (423),413988.0,False


In [5]:
# pivot the table to get sectors as columns with tax values by city
sectors_q2_2020 = pd.pivot_table(q2_2020,values = '2020Q2',index ='City', columns = 'Economic Sector (NAICS Code)', aggfunc='first')

# fill NAs with 0
sectors_q2_2020 = sectors_q2_2020.fillna(0)

In [7]:
sectors_q2_2020.columns

Index(['ACCOMMODATION (721)',
       'ADMIN. & SUPPORT & WASTE MANAG. & REMED. SERVICES (56)',
       'AGRICULTURE, FORESTRY, FISHING & HUNTING (11)',
       'ARTS, ENTERTAINMENT AND RECREATION (71)', 'CONSTRUCTION (23)',
       'EDUCATIONAL SERVICES (61)', 'FINANCE & INSURANCE (52)',
       'FOOD SERVICES & DRINKING PLACES (722)',
       'HEALTH CARE & SOCIAL ASSISTANCE (62)', 'INFORMATION (51)',
       'MANAGEMENT OF COMPANIES & ENTERPRISES (55)', 'MANUFACTURING (31-33)',
       'MINING, QUARRYING, & OIL & GAS EXTRACTION (21)',
       'OTHER SERVICES-EXCEPT PUBLIC ADMINISTRATION (81)',
       'PRIOR-PERIOD PAYMENTS & REFUNDS', 'PRIVATE MOTOR VEHICLE SALES',
       'PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES (54)',
       'PUBLIC ADMINISTRATION (92)', 'REAL ESTATE, RENTAL & LEASING (53)',
       'RETAIL-BUILD. MATERIAL, GARDEN EQUIP. & SUPPLIES DEALERS (444)',
       'RETAIL-CLOTHING & CLOTHING ACCESSORIES STORES (448)',
       'RETAIL-ELECTRONICS & APPLIANCE STORES (443)',
       

In [5]:
# fix column names
#new_names = [replace(item) for item in list(sectors_q2_2020.columns)]
new_names = ['L_ACCOMMODATION_20', 'S_ADMIN_SUPPORT_20','A_AG_WILDLIFE_20','L_CULTURAL_REC_20','C_CONSTRUCTION_20',
'E_EDUCATION_20','S_FINANCIAL_20','L_RSTRNT_BAR_20','H_HEALTH_CARE_20','S_IT_DATA_20','S_CORPORATE_MGMT_20',
'M_MANUFACTURING_20','E_EXTRACTION_20','O_OTHER_20','X_ADJUSTMENTS_20','R_AUTO_PRIVATE_20','S_PROF_TECH_SRV_20',
'G_GOVERNMENT_20','S_RENTAL_LEASING_20','R_BUILDING_SUPPLY_20','R_CLOTHING_20','R_ELECTRONICS_20','R_GROCERY_BEV_20',
'R_FURNITURE_20','R_GAS_STATIONS_20','R_GENERAL_RETAIL_20','R_HEALTH_RETAIL_20','R_OTHER_RETAIL_20','R_AUTO_RETAIL_20',
'R_NONSTORE_RETAIL_20','R_SPORT_HOBBY_20','L_SPECIAL_EVENT_20','W_DISTRIBUTION_20','X_UNKNOWN_20','U_UTILITIES_20',
'W_WHLSALE_DURABLE_20','S_WHLSLE_ETRADE_20','W_WHLSLE_NDURABLE_20']
sectors_q2_2020.columns = new_names

# get the total sales tax revenue
sectors_q2_2020['TOTAL_20'] = sectors_q2_2020[new_names].sum(axis=1)

sectors_q2_2020.head()

,L_ACCOMMODATION_20,S_ADMIN_SUPPORT_20,A_AG_WILDLIFE_20,L_CULTURAL_REC_20,C_CONSTRUCTION_20,E_EDUCATION_20,S_FINANCIAL_20,L_RSTRNT_BAR_20,H_HEALTH_CARE_20,S_IT_DATA_20,S_CORPORATE_MGMT_20,M_MANUFACTURING_20,E_EXTRACTION_20,O_OTHER_20,X_ADJUSTMENTS_20,R_AUTO_PRIVATE_20,S_PROF_TECH_SRV_20,G_GOVERNMENT_20,S_RENTAL_LEASING_20,R_BUILDING_SUPPLY_20,R_CLOTHING_20,R_ELECTRONICS_20,R_GROCERY_BEV_20,R_FURNITURE_20,R_GAS_STATIONS_20,R_GENERAL_RETAIL_20,R_HEALTH_RETAIL_20,R_OTHER_RETAIL_20,R_AUTO_RETAIL_20,R_NONSTORE_RETAIL_20,R_SPORT_HOBBY_20,L_SPECIAL_EVENT_20,W_DISTRIBUTION_20,X_UNKNOWN_20,U_UTILITIES_20,W_WHLSALE_DURABLE_20,S_WHLSLE_ETRADE_20,W_WHLSLE_NDURABLE_20,TOTAL_20
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,360525.0,625281.0,1750000.0,1132810.0,10855679.0,69947.0,1510886.0,25740513.0,368700.0,6514315.0,25000.0,3709354.0,60000.0,12382993.0,2622074.0,4500000.0,4496886.0,90000.0,3964900.0,29535937.0,5219505.0,3463441.0,4360567.0,2896641.0,3000000.0,45182823.0,5071921.0,7479307.0,109956893.0,19033396.0,7849449.0,50000.0,498043.0,36566.0,5750000.0,15336406.0,459562.0,11190788.0,357151108.0
Beaver City,988520.0,17144.0,400000.0,7000.0,29553.0,10000.0,457083.0,2263825.0,60000.0,495267.0,1000.0,1308568.0,0.0,1021247.0,142192.0,700000.0,153031.0,0.0,143519.0,1005793.0,102242.0,26765.0,2250000.0,9064.0,3750000.0,554323.0,108995.0,235502.0,828081.0,1871246.0,36255.0,35000.0,20000.0,3747.0,600000.0,413988.0,80000.0,146624.0,20275574.0
Blanding,408578.0,45672.0,1000.0,20000.0,305743.0,50000.0,17919.0,600000.0,60000.0,485187.0,5000.0,77542.0,0.0,363436.0,59705.0,900000.0,143740.0,500000.0,59894.0,2720256.0,80220.0,111496.0,2250000.0,3258.0,1500000.0,748656.0,73134.0,124955.0,394683.0,1473315.0,76601.0,0.0,20000.0,527.0,90000.0,308763.0,8000.0,232607.0,14319887.0
Bountiful,100000.0,975548.0,4000.0,585659.0,1647021.0,81571.0,1116141.0,11839024.0,935892.0,5678745.0,25000.0,2171121.0,5000.0,5958817.0,1671381.0,5500000.0,3301438.0,900000.0,2053241.0,1898525.0,2654922.0,3819206.0,24807045.0,1691416.0,2500000.0,2278386.0,2195173.0,3755609.0,58937846.0,17497067.0,2248783.0,150000.0,35000.0,16822.0,8500000.0,4763409.0,131656.0,641651.0,183072115.0
Brigham,655681.0,186299.0,3000.0,600000.0,1456710.0,28719.0,201424.0,6912003.0,297620.0,2767801.0,15000.0,9242535.0,1000.0,3281464.0,715733.0,3250000.0,796656.0,1000.0,1868664.0,2727400.0,440580.0,1228067.0,11722561.0,507641.0,3407129.0,628164.0,355527.0,896487.0,18008684.0,6288043.0,596162.0,30000.0,45507.0,3732.0,2750000.0,5838242.0,72838.0,243912.0,88071985.0


### Load Q2 2019 sales tax revenue data

In [6]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2019-02-quarterly-sales.xlsx'
q2_2019 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
q2_2019.drop(q2_2019.tail(2).index,inplace=True)

# forward fill values from merged cells
q2_2019['County'] = q2_2019['County'].fillna(method='ffill')
q2_2019['Location Code'] = q2_2019['Location Code'].fillna(method='ffill')
q2_2019['City'] = q2_2019['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
q2_2019.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
q2_2019['Rounded Up'] = q2_2019['Rounded Up'].map(crosswalk)

q2_2019.tail()

,County,Location Code,City,Economic Sector (NAICS Code),2019Q2,Rounded Up
2411,Weber County,29040.0,South Ogden,FOOD SERVICES & DRINKING PLACES (722),10298427.0,False
2412,Weber County,29040.0,South Ogden,OTHER SERVICES-EXCEPT PUBLIC ADMINISTRATION (81),1429711.0,False
2413,Weber County,29040.0,South Ogden,PRIVATE MOTOR VEHICLE SALES,1500000.0,True
2414,Weber County,29040.0,South Ogden,SPECIAL EVENT SALES,45000.0,True
2415,Weber County,29040.0,South Ogden,PRIOR-PERIOD PAYMENTS & REFUNDS,165743.0,False


In [7]:
# pivot the table to get sectors as columns with tax values by city
sectors_q2_2019 = pd.pivot_table(q2_2019,values = '2019Q2',index ='City', columns = 'Economic Sector (NAICS Code)', aggfunc='first')

# fill NAs with 0
sectors_q2_2019 = sectors_q2_2019.fillna(0)

# fix column names
#new_names = [replace(item) for item in list(sectors_q2_2019.columns)]
new_names = ['L_ACCOMMODATION_19', 'S_ADMIN_SUPPORT_19','A_AG_WILDLIFE_19','L_CULTURAL_REC_19','C_CONSTRUCTION_19',
'E_EDUCATION_19','S_FINANCIAL_19','L_RSTRNT_BAR_19','H_HEALTH_CARE_19','S_IT_DATA_19','S_CORPORATE_MGMT_19',
'M_MANUFACTURING_19','E_EXTRACTION_19','O_OTHER_19','X_ADJUSTMENTS_19','R_AUTO_PRIVATE_19','S_PROF_TECH_SRV_19',
'G_GOVERNMENT_19','S_RENTAL_LEASING_19','R_BUILDING_SUPPLY_19','R_CLOTHING_19','R_ELECTRONICS_19','R_GROCERY_BEV_19',
'R_FURNITURE_19','R_GAS_STATIONS_19','R_GENERAL_RETAIL_19','R_HEALTH_RETAIL_19','R_OTHER_RETAIL_19','R_AUTO_RETAIL_19',
'R_NONSTORE_RETAIL_19','R_SPORT_HOBBY_19','L_SPECIAL_EVENT_19','W_DISTRIBUTION_19','X_UNKNOWN_19','U_UTILITIES_19',
'W_WHLSALE_DURABLE_19','S_WHLSLE_ETRADE_19','W_WHLSLE_NDURABLE_19']
sectors_q2_2019.columns = new_names

# get the total sales tax revenue
sectors_q2_2019['TOTAL_19'] = sectors_q2_2019[new_names].sum(axis=1)

sectors_q2_2019.head()

,L_ACCOMMODATION_19,S_ADMIN_SUPPORT_19,A_AG_WILDLIFE_19,L_CULTURAL_REC_19,C_CONSTRUCTION_19,E_EDUCATION_19,S_FINANCIAL_19,L_RSTRNT_BAR_19,H_HEALTH_CARE_19,S_IT_DATA_19,S_CORPORATE_MGMT_19,M_MANUFACTURING_19,E_EXTRACTION_19,O_OTHER_19,X_ADJUSTMENTS_19,R_AUTO_PRIVATE_19,S_PROF_TECH_SRV_19,G_GOVERNMENT_19,S_RENTAL_LEASING_19,R_BUILDING_SUPPLY_19,R_CLOTHING_19,R_ELECTRONICS_19,R_GROCERY_BEV_19,R_FURNITURE_19,R_GAS_STATIONS_19,R_GENERAL_RETAIL_19,R_HEALTH_RETAIL_19,R_OTHER_RETAIL_19,R_AUTO_RETAIL_19,R_NONSTORE_RETAIL_19,R_SPORT_HOBBY_19,L_SPECIAL_EVENT_19,W_DISTRIBUTION_19,X_UNKNOWN_19,U_UTILITIES_19,W_WHLSALE_DURABLE_19,S_WHLSLE_ETRADE_19,W_WHLSLE_NDURABLE_19,TOTAL_19
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,700000.0,566647.0,1250000.0,1600920.0,8794650.0,99011.0,1790289.0,24651818.0,498511.0,9244154.0,20000.0,3599673.0,25000.0,11024408.0,3859810.0,3000000.0,2182874.0,35000.0,3830247.0,23085674.0,7224842.0,7042705.0,4026871.0,3546430.0,2887348.0,41378967.0,5619710.0,6320643.0,89772175.0,7550237.0,7817656.0,20000.0,600000.0,0.0,8250000.0,11443316.0,25000.0,7358148.0,310742734.0
Beaver City,1423694.0,31711.0,250000.0,7000.0,35000.0,15000.0,220378.0,2737418.0,50000.0,722352.0,1000.0,698857.0,0.0,889776.0,719732.0,500000.0,96364.0,0.0,100714.0,564664.0,58911.0,23509.0,1750000.0,4000.0,3500000.0,444185.0,72269.0,143038.0,615787.0,587176.0,23446.0,20000.0,4000.0,0.0,600000.0,524356.0,25000.0,203124.0,17662461.0
Blanding,1386710.0,48945.0,1000.0,90000.0,210655.0,250000.0,25496.0,700000.0,60000.0,483587.0,5000.0,164496.0,0.0,260314.0,189826.0,800000.0,61873.0,500000.0,93226.0,1948647.0,62024.0,39162.0,2000000.0,7000.0,1250000.0,634468.0,58423.0,125019.0,305191.0,656070.0,43412.0,0.0,25000.0,0.0,100000.0,598802.0,1000.0,402846.0,13588192.0
Bountiful,150000.0,2460364.0,2000.0,1105872.0,2074898.0,104656.0,1151100.0,13673812.0,987748.0,5176705.0,20000.0,2685170.0,10000.0,6990239.0,4448948.0,5250000.0,2157862.0,600000.0,2670871.0,1318455.0,2791903.0,3289392.0,20779545.0,1808556.0,2500000.0,1705541.0,1947420.0,3157634.0,53853811.0,7669928.0,1181965.0,90000.0,45000.0,0.0,7000000.0,3651271.0,9000.0,715392.0,165235058.0
Brigham,2250000.0,188781.0,3000.0,800000.0,1527976.0,58893.0,292577.0,7047410.0,499834.0,2584387.0,25000.0,6400432.0,1000.0,3013123.0,678568.0,2500000.0,391323.0,1000.0,1731936.0,1916996.0,392600.0,1059660.0,9955541.0,529234.0,3500000.0,1045589.0,485876.0,1491898.0,15865705.0,3295918.0,534664.0,90000.0,70000.0,0.0,2500000.0,5046056.0,10000.0,320490.0,78105467.0


### Calculate the differences 
*2020 - 2019 = Diff*

In [8]:
# merge the 2019-2020 difference with the sdf
sectors_q2_diff = pd.DataFrame(sectors_q2_2020.values - sectors_q2_2019.values)

# add back the column names
new_names = ['L_ACCOMMODATION_D20', 'S_ADMIN_SUPPORT_D20','A_AG_WILDLIFE_D20','L_CULTURAL_REC_D20','C_CONSTRUCTION_D20',
'E_EDUCATION_D20','S_FINANCIAL_D20','L_RSTRNT_BAR_D20','H_HEALTH_CARE_D20','S_IT_DATA_D20','S_CORPORATE_MGMT_D20',
'M_MANUFACTURING_D20','E_EXTRACTION_D20','O_OTHER_D20','X_ADJUSTMENTS_D20','R_AUTO_PRIVATE_D20','S_PROF_TECH_SRV_D20',
'G_GOVERNMENT_D20','S_RENTAL_LEASING_D20','R_BUILDING_SUPPLY_D20','R_CLOTHING_D20','R_ELECTRONICS_D20','R_GROCERY_BEV_D20',
'R_FURNITURE_D20','R_GAS_STATIONS_D20','R_GENERAL_RETAIL_D20','R_HEALTH_RETAIL_D20','R_OTHER_RETAIL_D20','R_AUTO_RETAIL_D20',
'R_NONSTORE_RETAIL_D20','R_SPORT_HOBBY_D20','L_SPECIAL_EVENT_D20','W_DISTRIBUTION_D20','X_UNKNOWN_D20','U_UTILITIES_D20',
'W_WHLSALE_DURABLE_D20','S_WHLSLE_ETRADE_D20','W_WHLSLE_NDURABLE_D20', 'TOTAL_D20']

sectors_q2_diff.columns = new_names

# add back cities as index
sectors_q2_diff['City'] = sectors_q2_2020.index
sectors_q2_diff = sectors_q2_diff.set_index('City')

#check table
sectors_q2_diff.head(10)

,L_ACCOMMODATION_D20,S_ADMIN_SUPPORT_D20,A_AG_WILDLIFE_D20,L_CULTURAL_REC_D20,C_CONSTRUCTION_D20,E_EDUCATION_D20,S_FINANCIAL_D20,L_RSTRNT_BAR_D20,H_HEALTH_CARE_D20,S_IT_DATA_D20,S_CORPORATE_MGMT_D20,M_MANUFACTURING_D20,E_EXTRACTION_D20,O_OTHER_D20,X_ADJUSTMENTS_D20,R_AUTO_PRIVATE_D20,S_PROF_TECH_SRV_D20,G_GOVERNMENT_D20,S_RENTAL_LEASING_D20,R_BUILDING_SUPPLY_D20,R_CLOTHING_D20,R_ELECTRONICS_D20,R_GROCERY_BEV_D20,R_FURNITURE_D20,R_GAS_STATIONS_D20,R_GENERAL_RETAIL_D20,R_HEALTH_RETAIL_D20,R_OTHER_RETAIL_D20,R_AUTO_RETAIL_D20,R_NONSTORE_RETAIL_D20,R_SPORT_HOBBY_D20,L_SPECIAL_EVENT_D20,W_DISTRIBUTION_D20,X_UNKNOWN_D20,U_UTILITIES_D20,W_WHLSALE_DURABLE_D20,S_WHLSLE_ETRADE_D20,W_WHLSLE_NDURABLE_D20,TOTAL_D20
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,-339475.0,58634.0,500000.0,-468110.0,2061029.0,-29064.0,-279403.0,1088695.0,-129811.0,-2729839.0,5000.0,109681.0,35000.0,1358585.0,-1237736.0,1500000.0,2314012.0,55000.0,134653.0,6450263.0,-2005337.0,-3579264.0,333696.0,-649789.0,112652.0,3803856.0,-547789.0,1158664.0,20184718.0,11483159.0,31793.0,30000.0,-101957.0,36566.0,-2500000.0,3893090.0,434562.0,3832640.0,46408374.0
Beaver City,-435174.0,-14567.0,150000.0,0.0,-5447.0,-5000.0,236705.0,-473593.0,10000.0,-227085.0,0.0,609711.0,0.0,131471.0,-577540.0,200000.0,56667.0,0.0,42805.0,441129.0,43331.0,3256.0,500000.0,5064.0,250000.0,110138.0,36726.0,92464.0,212294.0,1284070.0,12809.0,15000.0,16000.0,3747.0,0.0,-110368.0,55000.0,-56500.0,2613113.0
Blanding,-978132.0,-3273.0,0.0,-70000.0,95088.0,-200000.0,-7577.0,-100000.0,0.0,1600.0,0.0,-86954.0,0.0,103122.0,-130121.0,100000.0,81867.0,0.0,-33332.0,771609.0,18196.0,72334.0,250000.0,-3742.0,250000.0,114188.0,14711.0,-64.0,89492.0,817245.0,33189.0,0.0,-5000.0,527.0,-10000.0,-290039.0,7000.0,-170239.0,731695.0
Bountiful,-50000.0,-1484816.0,2000.0,-520213.0,-427877.0,-23085.0,-34959.0,-1834788.0,-51856.0,502040.0,5000.0,-514049.0,-5000.0,-1031422.0,-2777567.0,250000.0,1143576.0,300000.0,-617630.0,580070.0,-136981.0,529814.0,4027500.0,-117140.0,0.0,572845.0,247753.0,597975.0,5084035.0,9827139.0,1066818.0,60000.0,-10000.0,16822.0,1500000.0,1112138.0,122656.0,-73741.0,17837057.0
Brigham,-1594319.0,-2482.0,0.0,-200000.0,-71266.0,-30174.0,-91153.0,-135407.0,-202214.0,183414.0,-10000.0,2842103.0,0.0,268341.0,37165.0,750000.0,405333.0,0.0,136728.0,810404.0,47980.0,168407.0,1767020.0,-21593.0,-92871.0,-417425.0,-130349.0,-595411.0,2142979.0,2992125.0,61498.0,-60000.0,-24493.0,3732.0,250000.0,792186.0,62838.0,-76578.0,9966518.0
Cedar City,-3002509.0,367130.0,8530.0,-186282.0,740470.0,-350973.0,-111789.0,-744532.0,106396.0,75992.0,-5000.0,3067716.0,-19000.0,13652.0,1795465.0,1500000.0,1004478.0,-50000.0,41261.0,8422563.0,-770723.0,107977.0,3575821.0,1016051.0,1312821.0,6380002.0,203473.0,1471677.0,3723202.0,7793295.0,1615163.0,750000.0,-1522426.0,13895.0,-250000.0,2667554.0,264017.0,-106028.0,40919339.0
Centerville,-5000.0,50243.0,0.0,-10000.0,-484578.0,3773.0,-159829.0,-277358.0,-53637.0,-2228983.0,-275000.0,-278122.0,-2000.0,-169211.0,-102928.0,750000.0,436386.0,-15000.0,107902.0,4043706.0,8082.0,-501098.0,672530.0,488854.0,0.0,963036.0,52319.0,-638112.0,4748383.0,3756985.0,454986.0,450000.0,10000.0,7713.0,0.0,1068946.0,48099.0,-53460.0,12867627.0
Clearfield,-50000.0,96076.0,1000.0,-250000.0,163890.0,-5104.0,79958.0,-361897.0,16337.0,731313.0,400000.0,-1219753.0,10000.0,-300133.0,8493686.0,250000.0,1372909.0,-180000.0,-168479.0,893720.0,399082.0,-92258.0,943586.0,81360.0,829938.0,411900.0,26805.0,1122579.0,1186290.0,7057602.0,248992.0,193000.0,-15219.0,7998.0,-1000000.0,2164321.0,149934.0,-127616.0,23561817.0
Cottonwood Heights,-686505.0,-29231.0,3000.0,-40000.0,180454.0,-13645.0,514343.0,-4758782.0,-138454.0,1292468.0,-100000.0,-1041200.0,-56000.0,-7233.0,-4342114.0,0.0,2554221.0,-150000.0,-477113.0,3021231.0,-1505248.0,-694143.0,1882358.0,178308.0,250000.0,4650983.0,390525.0,182014.0,819.0,7969657.0,110185.0,0.0,35000.0,27932.0,-1500000.0,311197.0,81283.0,122314.0,8218624.0


In [9]:
# Check join output shape
print(sectors_q2_2019.shape)
print(sectors_q2_2020.shape)
print(sectors_q2_diff.shape)

(66, 39)
(66, 39)
(66, 39)


### Load municipalities and townships shapefile and join to formatted sales tax data

In [10]:
# load cities shapefile into pandas spatial dataframe
cities_shp = '.\\Inputs\\Cities.shp'
cities_sdf = pd.DataFrame.spatial.from_featureclass(cities_shp)

# Rename columns
cities_sdf.columns = ['FID', 'NAME', 'SOURCE', 'SALESTAXID', 'POPLASTCEN', 'POPLASTEST',
       'AREA_SQMI', 'SHAPE']

In [11]:
# merge the 2020 sdf with the pivoted data
cities_sdf2 = cities_sdf.merge(sectors_q2_2019, left_on= "NAME", right_on="City", how="inner")
cities_sdf2 = cities_sdf2.merge(sectors_q2_2020, left_on= "NAME", right_on="City", how="inner")
cities_sdf2 = cities_sdf2.merge(sectors_q2_diff, left_on= "NAME", right_on="City", how="inner")
cities_sdf2.tail(10)


,FID,NAME,SOURCE,SALESTAXID,POPLASTCEN,POPLASTEST,AREA_SQMI,SHAPE,L_ACCOMMODATION_19,S_ADMIN_SUPPORT_19,A_AG_WILDLIFE_19,L_CULTURAL_REC_19,C_CONSTRUCTION_19,E_EDUCATION_19,S_FINANCIAL_19,L_RSTRNT_BAR_19,H_HEALTH_CARE_19,S_IT_DATA_19,S_CORPORATE_MGMT_19,M_MANUFACTURING_19,E_EXTRACTION_19,O_OTHER_19,X_ADJUSTMENTS_19,R_AUTO_PRIVATE_19,S_PROF_TECH_SRV_19,G_GOVERNMENT_19,S_RENTAL_LEASING_19,R_BUILDING_SUPPLY_19,R_CLOTHING_19,R_ELECTRONICS_19,R_GROCERY_BEV_19,R_FURNITURE_19,R_GAS_STATIONS_19,R_GENERAL_RETAIL_19,R_HEALTH_RETAIL_19,R_OTHER_RETAIL_19,R_AUTO_RETAIL_19,R_NONSTORE_RETAIL_19,R_SPORT_HOBBY_19,L_SPECIAL_EVENT_19,W_DISTRIBUTION_19,X_UNKNOWN_19,U_UTILITIES_19,W_WHLSALE_DURABLE_19,S_WHLSLE_ETRADE_19,W_WHLSLE_NDURABLE_19,TOTAL_19,L_ACCOMMODATION_20,S_ADMIN_SUPPORT_20,A_AG_WILDLIFE_20,L_CULTURAL_REC_20,C_CONSTRUCTION_20,E_EDUCATION_20,S_FINANCIAL_20,L_RSTRNT_BAR_20,H_HEALTH_CARE_20,S_IT_DATA_20,S_CORPORATE_MGMT_20,M_MANUFACTURING_20,E_EXTRACTION_20,O_OTHER_20,X_ADJUSTMENTS_20,R_AUTO_PRIVATE_20,S_PROF_TECH_SRV_20,G_GOVERNMENT_20,S_RENTAL_LEASING_20,R_BUILDING_SUPPLY_20,R_CLOTHING_20,R_ELECTRONICS_20,R_GROCERY_BEV_20,R_FURNITURE_20,R_GAS_STATIONS_20,R_GENERAL_RETAIL_20,R_HEALTH_RETAIL_20,R_OTHER_RETAIL_20,R_AUTO_RETAIL_20,R_NONSTORE_RETAIL_20,R_SPORT_HOBBY_20,L_SPECIAL_EVENT_20,W_DISTRIBUTION_20,X_UNKNOWN_20,U_UTILITIES_20,W_WHLSALE_DURABLE_20,S_WHLSLE_ETRADE_20,W_WHLSLE_NDURABLE_20,TOTAL_20,L_ACCOMMODATION_D20,S_ADMIN_SUPPORT_D20,A_AG_WILDLIFE_D20,L_CULTURAL_REC_D20,C_CONSTRUCTION_D20,E_EDUCATION_D20,S_FINANCIAL_D20,L_RSTRNT_BAR_D20,H_HEALTH_CARE_D20,S_IT_DATA_D20,S_CORPORATE_MGMT_D20,M_MANUFACTURING_D20,E_EXTRACTION_D20,O_OTHER_D20,X_ADJUSTMENTS_D20,R_AUTO_PRIVATE_D20,S_PROF_TECH_SRV_D20,G_GOVERNMENT_D20,S_RENTAL_LEASING_D20,R_BUILDING_SUPPLY_D20,R_CLOTHING_D20,R_ELECTRONICS_D20,R_GROCERY_BEV_D20,R_FURNITURE_D20,R_GAS_STATIONS_D20,R_GENERAL_RETAIL_D20,R_HEALTH_RETAIL_D20,R_OTHER_RETAIL_D20,R_AUTO_RETAIL_D20,R_NONSTORE_RETAIL_D20,R_SPORT_HOBBY_D20,L_SPECIAL_EVENT_D20,W_DISTRIBUTION_D20,X_UNKNOWN_D20,U_UTILITIES_D20,W_WHLSALE_DURABLE_D20,S_WHLSLE_ETRADE_D20,W_WHLSLE_NDURABLE_D20,TOTAL_D20
55,222,Syracuse,Cities,049,25113,30400,10.193900,"{'rings': [[[411406.60675760085, 4549994.30436...",40000.0,91580.0,1000.0,1250000.0,204550.0,20000.0,531340.0,4888061.0,50000.0,2771091.0,8000.0,435243.0,15000.0,1387300.0,778008.0,4250000.0,326002.0,3000.0,797454.0,591449.0,345673.0,698056.0,8510522.0,9829414.0,500000.0,17991753.0,355277.0,360256.0,477965.0,4608070.0,125271.0,70000.0,15000.0,0.0,3250000.0,813132.0,2000.0,218041.0,66609508.0,35000.0,486195.0,3000.0,800000.0,1640469.0,26514.0,423856.0,5112573.0,70000.0,2782220.0,15000.0,806783.0,5000.0,1435603.0,565886.0,5500000.0,823093.0,2000.0,1095829.0,846234.0,682872.0,1039001.0,11360791.0,9055656.0,500000.0,21412672.0,445899.0,694081.0,664686.0,12080397.0,288788.0,250000.0,45000.0,-2375.0,3500000.0,1843551.0,104216.0,401312.0,86841802.0,-5000.0,394615.0,2000.0,-450000.0,1435919.0,6514.0,-107484.0,224512.0,20000.0,11129.0,7000.0,371540.0,-10000.0,48303.0,-212122.0,1250000.0,497091.0,-1000.0,298375.0,254785.0,337199.0,340945.0,2850269.0,-773758.0,0.0,3420919.0,90622.0,333825.0,186721.0,7472327.0,163517.0,180000.0,30000.0,-2375.0,250000.0,1030419.0,102216.0,183271.0,20232294.0
56,224,Taylorsville,Cities,142,59989,60192,10.839700,"{'rings': [[[422191.826826254, 4504303.3642982...",300000.0,511515.0,15000.0,2107794.0,1455466.0,161570.0,1788978.0,28052721.0,105335.0,8700270.0,25000.0,2203354.0,5000.0,2431780.0,2938768.0,4500000.0,1445801.0,3000.0,2161201.0,563109.0,5900761.0,1738356.0,14885981.0,369133.0,4250000.0,25725971.0,1146084.0,4688743.0,6753518.0,6473158.0,6072459.0,150000.0,40000.0,0.0,7500000.0,2760698.0,195299.0,435741.0,148561564.0,300000.0,578798.0,10000.0,1492444.0,1298151.0,74530.0,2121107.0,24753763.0,133238.0,6367201.0,20000.0,2556912.0,6000.0,2064008.0,1429666.0,5000000.0,2857465.0,2000.0,1882288.0,588691.0,3770433.0,2974755.0,15555818.0,506793.0,4750000.0,28265174.0,1465501.0,5169

In [12]:
# Confirm join count, Salt Lake County (Unincorporated) should be the only one not joined
l1 = list(q2_2020['City'].value_counts().index)
l2 = list(cities_sdf['NAME'].value_counts().index)
list(set(l1) - set(l2))

['Salt Lake County (Unincorporated)']

In [13]:
# Create file gdb and export sdf to feature class
outputs = '.\\Outputs'

gdb = os.path.join(outputs, "taxable_sales.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "taxable_sales.gdb")

cities_sdf2.spatial.to_featureclass(location=os.path.join(outputs, "taxable_sales.gdb","taxable_sales_utah_q2"))

'E:\\Projects\\UT_Sales_Tax_Analysis\\Outputs\\taxable_sales.gdb\\taxable_sales_utah_q2'

### Analysis

In [14]:
# Get year specific column names
standard_cols = ['fid','name','source','salestaxid','poplastcen','poplastcen','area_sqmi', 'SHAPE']
names_19 = standard_cols + [col for col in cities_sdf2.columns if '_19' in col]
names_20 = standard_cols + [col for col in cities_sdf2.columns if '_20' in col]
names_D20 = standard_cols + [col for col in cities_sdf2.columns if '_d20' in col]

# subset columns by time
cities_q2_19 = cities_sdf2[names_19]
cities_q2_20 = cities_sdf2[names_20]
cities_q2_d20 = cities_sdf2[names_D20]


In [15]:
# 10 cities with smallest difference of revenue
cities_q2_d20[['name', 'poplastcen', 'area_sqmi', 'total_d20']].sort_values('total_d20', ascending=False).head(10)

,name,poplastcen,poplastcen,area_sqmi,total_d20
62,West Jordan,105891,105891,32.306999,91207296.0
54,St George,76548,76548,77.209900,84029566.0
51,Spanish Fork,36232,36232,16.249701,57141263.0
57,Tooele City,32629,32629,24.087099,48598585.0
63,West Valley City,133796,133796,35.799702,46907870.0
0,American Fork,27413,27413,11.256600,46408374.0
20,Layton,69457,69457,22.576200,42043505.0
5,Cedar City,30139,30139,36.053398,40919339.0
23,Logan,49612,49612,18.485800,40702065.0
34,Orem,92993,92993,18.628000,40443599.0


In [16]:
# 10 cities with largest difference of revenue
cities_q2_d20[['name', 'poplastcen', 'area_sqmi', 'total_d20']].sort_values('total_d20', ascending=True).head(10)

,name,poplastcen,poplastcen,area_sqmi,total_d20
45,Salt Lake City,190749,190749,110.734001,-217551883.0
27,Moab,5124,5124,4.802460,-43345377.0
35,Park City,7709,7709,20.341700,-42810440.0
30,Murray,47311,47311,12.293700,-40848514.0
48,South Jordan,51057,51057,22.263201,-25277957.0
52,Springdale,569,569,4.594880,-21011489.0
12,Farmington,18752,18752,10.035100,-17939769.0
43,Roosevelt,6260,6260,6.757070,-9635486.0
17,Kanab,4413,4413,14.473200,-6206268.0
59,Vernal,9460,9460,4.632710,-3023499.0


#### Some industries of interest:
- RETAIL-GASOLINE STATIONS (447)
- RETAIL-FOOD & BEVERAGE STORES (445)
- REAL ESTATE, RENTAL & LEASING (53)
- ARTS, ENTERTAINMENT AND RECREATION (71)
- ACCOMMODATION (721)